# Evolutionary algorithms analysis

Complete run of the evolutionary algorithms analysis. This notebook is used to run different evolutionary algorithms and test them on benchmark functions from benchamrk_function.py file. The results are saved in the results folder. 

Analysis is done via average ranks, with specific notes on outliers.

In [1]:
# Evo algorithms setup

# Imports
import source.evolutionary_algorithms as ea
import numpy as np
import source.helpers as hp
import importlib
import inspect
import time
import sys
import os

# Load benchmark functions
module_path = os.path.abspath(os.path.join('./source'))
sys.path.append(module_path)
benchmarks = importlib.import_module('benchmark_functions')
functions = [obj for _, obj in inspect.getmembers(benchmarks) if inspect.isfunction(obj)]

# Hyperparameters
DIMENSIONS = [2, 10, 30]

lower_bound = -100.0
upper_bound = 100.0

POPULATION_SIZES = [10, 20, 50]
FUNCTION_EVALUATIONS = 2000
RESULTS = dict()
REPEATS = 30

## Running ea

Run ea for benchmark functions and write results to global variables.


In [2]:
USE_PERSISTENT_DATA = True
FILENAME = f"results/full_run_1704636408.496384.json" # Rename this in case of 

if USE_PERSISTENT_DATA:
    RESULTS = hp.load_results(FILENAME)
    print("Loaded results from file")
else:
    for i, dim in enumerate(DIMENSIONS):
        BOUNDS = np.array([[lower_bound, upper_bound]] * DIMENSIONS[i])
        pop_size = POPULATION_SIZES[i]
        evaluations = dim * FUNCTION_EVALUATIONS
        dimension_results = []
        print(f"Calculating dimension {dim} - ", end="")
        counter = 0

        for function in functions:
            runs = []
            counter += 1
            
            for r in range(REPEATS):
                row = [0, 0, 0, 0, 0]
                row[0] = ea.differential_evolution(function, BOUNDS, pop_size, max_evaluations=evaluations, F=0.8, strategy="rand/1/bin")[1]
                row[1] = ea.differential_evolution(function, BOUNDS, pop_size, max_evaluations=evaluations, F=0.5, strategy="best/1/bin")[1]
                row[2] = ea.particle_swarm_optimization(function, BOUNDS, pop_size, max_evaluations=evaluations)[1]
                row[3] = ea.soma_ato(function, BOUNDS, pop_size, max_evaluations=evaluations)[1]
                row[4] = ea.soma_ata(function, BOUNDS, pop_size, max_evaluations=evaluations)[1]
                runs.append(row)

            print("#", end="")
            if counter % 5 == 0:
                print("|", end="")

            dimension_results.append(runs)
        
        RESULTS[dim] = dimension_results
        print("")
        
    if REPEATS == 30:
        RUN_NAME = "full"
    else:
        RUN_NAME = "partial"

    hp.save_results(RESULTS, f"results/{RUN_NAME}_run_{time.time()}.json")


Calculating dimension 2 - #####|#####|#####|#####|#####|
Calculating dimension 10 - #####|#####|#####|#####|#####|
Calculating dimension 30 - #####|#####|#####|#####|#####|


## Analyze results

Legend/note:
- **Rank**: evolutionary algorithm's average rank for benchmark function across number of runs (REPEATS)
- **Rank Avg.**: distance between Rank and Total Avg. Rank
- **Total Avg. Rank**: average rank of evolutionary algorithm across all benchmark functions
- **Chi square**: used to test if Rank Avg. is significantly different from Average
  - if p-value is less than 0.05 then Rank Avg. is significantly different from Average

In [3]:
for dim in RESULTS:
    print(f"\nResults for {dim} dimensions:\n")
    ranking_table = []
    hp.table_header()
    result_dict = dict()
    for i, fun in enumerate(RESULTS[dim]):
        res = np.zeros(5)
        for run in fun:
            res += hp.rank_array(run)
        res /= REPEATS 
        result_dict[functions[i]._custom_name] = res
        ranking_table.append(res)

    average_ranks = np.mean(ranking_table, axis=0)
    avg_distance = 0
    
    for pair in result_dict:
        result_rank = result_dict[pair]
        distance = hp.euclidean_distance(result_rank, average_ranks)
        avg_distance += distance
        hp.table_row(pair, result_rank, distance)

    avg_distance /= len(result_dict)
    
    hp.table_footer(average_ranks, avg_distance)
    chi, p = hp.friedman_test(ranking_table)
    print("\nFriedman test:")
    print("Chi-square: {:>14.6}\nP-value: {:>18.6}".format(chi, p))

    print("\n\n")


Results for 2 dimensions:

--------------------------------------------------------------------------------------------------
| Evo. Algorithm → | DE Rand 1  | DE Best 1  |    PSO     |  SOMA ato  |  SOMA ata  | Rank Avg.  |
| Benchmark ↓      |   (Rank)   |   (Rank)   |   (Rank)   |   (Rank)   |   (Rank)   |   (Diff)   |
--------------------------------------------------------------------------------------------------
| Ackley 1st       |    1.80    |    3.47    |    2.90    |    3.27    |    4.50    |    1.05    |
| Ackley 1st Alt.  |    3.13    |    4.90    |    1.37    |    3.23    |    2.37    |    2.31    |
| Alpine 1st       |    1.47    |    3.13    |    2.33    |    3.40    |    4.73    |    1.45    |
| Alpine 2nd       |    2.23    |    4.00    |    3.40    |    2.63    |    2.77    |    1.55    |
| Csendes          |    1.47    |    2.17    |    2.60    |    3.77    |    5.00    |    2.32    |
| Custom 1 (LLM)   |    1.47    |    4.10    |    3.47    |    2.50    |    3.70 

## Analysis of results
*(based on data from results/full_run_1704636408.496384.json)*

### Biggest differences
Note: Biggest differences are calculated as difference between Rank Avg. and Total Avg. Rank. This is done to find algorithms that are good at some functions, but bad at others. Greatest differences appear to be in very difficult functions. Those with many sharp hills and pits that have similar values for many minima tend to favor strategic local exploration. Others with hard to find global minima require more stochastic "exploration" and favored DE.

#### Dimension 2

##### Csendes

Csendes bowllike shape is easy to traverse and offers single-global minima. This makes it easy for algorithms to converge to global minima. Although, there is not enough steps for more complex algorithms to outperform simpler ones. This is why DE rand/1/bin and DE best/1/bin are able to outperform PSO and SOMAs

##### Svanda 1st 

This is a difficult function, with many local minima and maxima. Steep climbs and drops with half-pits and half-hills make it difficult for algorithms to converge to global minima. This is why PSO and SOMAs are able to outperform DE rand/1/bin and DE best/1/bin. Especially PSO did well here, as it was able to commit around minima in few steps available, being the most determininstic.

##### Svanda 3rd

Wave like shape with many local minima and maxima provide difficult terrain to functions that are prone to converge to local minima. Reasoning here is similar to Svanda 1st, as is function shape.

##### Svanda 5th

Function's shape is set of sporadic hills and pits, with many local minima. Around the center is weaker area, better values on sides. With few steps available, attempting pseudo-random exploration is not enough. This is why similar reasoning as in Svanda 1st and 3rd works here, as PSO did undisputably best.

##### Schwefel

Another function with multiple local minima, however way less steep climbs and drops than in previously mentioned functions. This made it easier for strategies with swarm like and migration like behavior to converge around minima. This is why PSO and SOMAs did well, with PSO being the best, but only closely.

##### Quartic

Quartic function is very similar to Csendes. Thus essentially the same reasoning applies as results themselves are very similar to those of Csendes.

#### Dimension 10

##### Svanda 1st

As mentioned in dimension 2, functions shape is difficult to traverse. This is why PSO does best, followed by SOMAs.

##### Svanda 2nd 

This function provides spiky terrain with major drop near the centre. Each pit is preceded by a hill, bigger the pit, bigger the hill. While many local minima are present, they are nowhere near as valuable as minimas around the centre. With problems increased complexity in dimension 10, DE rand/1/bin and DE best/1/bin are able to outperform PSO and SOMAs. This was due to enough steps being present with great value in finding minimas near the centre.

##### Svanda 3rd 

As mentioned in dimension 2, functions shape is difficult to traverse. This is why PSO does best, followed by SOMAs. Many local minima with a many of them having good values make more of them worth while to explore around.

##### Schwefel

Schwefel relatively non steep hills and pits that provide some difficulty traversing. Overall shape makes it likely to get into pit with good local minima when first traversing, alowing functions like PSO and SOMAs to then explore around the best member, thus finding great values.

##### Ackley 1st Altered

Very difficult function to traverse. Apart from PSO, every other functions exploration strategy was simply not efficient enough. PSO was able to traverse this terrain well, commit to minima and explore around it.

#### Dimension 30

##### Svanda 1st 

As mentioned in previous dimension, with some slight differences. With increased complexity, DE best/1/bin outperformed SOMA ata. This is likely due to steps available not being enough as ata's less efficient exploration strategy is not capable of finding good minima in time. DE best/1/bin with its more efficient yet more random strategy was able to commit better to exploration around minima.

##### Svanda 2nd

As mentioned in previous dimension, with some slight differences. With increased complexity, DE best/1/bin is clearly outperforming rand/1/bin strategy. With space being even more complex, having strategy for DE with some commitment to exploration in direction of current best member is very valuable.

##### Svanda 3rd 

Essentially the same reasoning as in dimension 2 and 10.

##### Zakharov

Interesting result. Zakharov is seemingly very simple function, akin to those like Quartic and De Jong 1st. Yet its U-like shape proposes different challenges. It seems, that mainly stochastic DE rand/1/bin was not enough (for dimension 30, funnily it seems that dim 10 was not that complex and it did best there). DE best/1/bin is by undisputably best here, supposedly mainly due to its fastest exploration, some directionality and some commitment to exploration around best member. Quick look at result values shows that other algorithms weren't able to get nowhere near the low value plane of function.

Taking account of all dimensions and results, this function doesn't behave as expected.

##### Ackley 1st Altered

Similar reasoning as in dimension 10, PSO's volatile, but targeted exploration strategy was able to commit to minima. Being able to explore in quite the radius at first, then precisely around the best thanks to particles' velocity preservation (from traveled distance) resulted being best strategy out of available.


